In [ ]:
#r "nuget: ARCExpect, 0.0.0-preview0"
#r "nuget: ARCTokenization"
#r "nuget: Graphoscope, 0.6.0-preview.1"
#r "nuget: FSharpAux, 2.0.0"
#r "nuget: Cytoscape.NET.Interactive"

open ARCTokenization
open ARCExpect
open Graphoscope
open FSharpAux
open Cytoscape.NET.Interactive

In [ ]:
// put in the path to your ARC's Investigation file here:

let path = @""

In [ ]:
let iparams = ARCTokenization.Investigation.parseMetadataSheetFromFile path

let onto = ARCTokenization.Terms.InvestigationMetadata.ontology

let ontoGraph = OboGraph.ontologyToFGraphByName onto

let ipsAdded = ARCGraph.addMissingTerms onto iparams

let partitionedIps = Seq.groupWhen (ARCGraph.isHeader ontoGraph) ipsAdded

let partitionallyFilledIps = partitionedIps |> Seq.map (ARCGraph.addMissingTermsInGroup ontoGraph)

let groupedIps = partitionallyFilledIps |> Seq.map ARCGraph.groupTerms

let matchedIps = groupedIps |> Seq.map (ARCGraph.matchTerms onto)

let subgraphs = Seq.map (ARCGraph.constructIntermediateMetadataSubgraph ontoGraph) matchedIps

let filledSubgraphs = Seq.map (fst >> ARCGraph.addEmptyIpsToNodeData) subgraphs

let splitSubgraphs = Seq.map ARCGraph.splitMetadataSubgraph filledSubgraphs

let filledLists = Seq.map ARCGraph.metadataSubgraphToList splitSubgraphs

In [ ]:
let contactsGraph =
    splitSubgraphs
    |> Seq.item 3
    |> ARCGraph.Visualization.isaSplitGraphToFullCyGraph

contactsGraph